In [185]:
# system tools
import warnings
import json
import sys
import string

# data cleaning + analysis tools
import pandas as pd
import datetime as dt
import numpy as np
import re

#nltk tools
import lda #Latent Dirichlet Allocation (create topics)
import gensim
from gensim import corpora, models #for constructing document term matrix
#from stop_words import get_stop_words
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

#set notebook preferences
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

### Import JSON file with city metadata - including which cities have sufficient Public Record Request (PRR) data for analysis

In [2]:
json_file = '../data/cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

###  Create dataframe of PRR data for all relevant cities

In [92]:
data_raw = pd.DataFrame(columns = ['city', 'month_year', 'Summary'])
city_list = []
for key, value in md.items():
    city = value['name']
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    if value["desc"] == "Y":
        try:
            df = pd.read_csv(filepath)
        except:
            try:
                df = pd.read_csv(filepath, encoding='mac_roman')
            except:
                continue
        print(key)
        name = key.split(' ')
        city_list.append(name[0].lower())
    else:
        continue
    
    try:
        df['Create Date'] = pd.to_datetime(df['Create Date'])
    except:
        df['New'] = pd.to_datetime(df['Create Date'].apply(lambda x: re.findall('^\S*', x)[0]))
        df.drop(columns=['Create Date'], inplace = True)
        df.rename(index=str, columns={"New": "Create Date"}, inplace = True)

    df['month_year'] = df['Create Date'].dt.to_period('M')
    
    mc = df[['month_year', 'Summary']]
    mc['city'] = city
    
    data_raw = pd.concat([data_raw, mc])

Arlington city
Asheville city
Bainbridge Island city
Boulder County
Cathedral City city
Dayton city
Denton city
Everett city
Fort Collins city
Greensboro city
Hayward city
Kirkland city
Las Cruces city
Lynnwood city
Mercer Island city
Miami city
Middleborough town
New Orleans city
Oakland city
Oklahoma City city
Olympia city
Palo Alto city
Peoria city
Pullman city
Rancho Cucamonga city
Redmond city
Renton city
Sacramento city
San Francisco city
Tukwila city
Vallejo city
West Sacramento city
Winchester city


#### We can see the raw data below. Our raw dataset includes 86,416 PRRs from 33 different cities

In [110]:
data_raw.head()

,index,Summary,city,month_year
0,0,We are working with an engineering firm on an ...,Arlington,2018-06
1,1,Need copies of contracts and all related docum...,Arlington,2018-06
2,2,"Copies of Building Permits of $5,000 valuation...",Arlington,2018-06
3,3,police report filed to an officer against Wayn...,Arlington,2018-06
4,4,"Email Communications between Stephanie Shook, ...",Arlington,2018-06


In [93]:
data_raw.shape

(86416, 3)

In [81]:
len(data_raw.city.unique())

33

In [94]:
data_raw.index = pd.RangeIndex(len(data_raw.index))

In [98]:
data_raw.reset_index(inplace=True)

### Create dataframe for cleaning by removing null summaries

In [189]:
data = data_raw.dropna(subset=['Summary'])

#### Function to convert nltk part of speech tags to wordnet tags (we use this to stem the words in data cleaning below):

In [171]:
def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

## Clean PRR data to prepare for LDA analysis

In [190]:
# Turn to lowercase
data.Summary = data.Summary.str.lower()

# Remove all punctuation
translator = str.maketrans('','', string.punctuation)
data.Summary = data.Summary.str.translate(translator)

# Remove all city names
for city in city_list:
    data.Summary = data.Summary.str.replace(city, '')
    
# Remove digits
dig_translator = str.maketrans('','', string.digits)
data.Summary = data.Summary.str.translate(dig_translator)

#remove empty strings, stopwords and stem
stop_words = set(stopwords.words('english'))
lmtzr = WordNetLemmatizer()
data['token'] = data['Summary'].apply(lambda x: nltk.word_tokenize(x))
data['lemma'] = data['token'].apply(lambda x: nltk.pos_tag(x))
data['mash'] = data['lemma'].apply(lambda x: [lmtzr.lemmatize(i[0], get_wordnet_pos(i[1])) for i in x if len(i[0]) > 0 and i[0] not in stop_words])

# Remove whitespace
wsp_translator = str.maketrans('','', string.whitespace)
data['mash'] = data['mash'].apply(lambda x: [i.translate(wsp_translator) for i in x])

# Remove empty lists
data['mash_len'] = data['mash'].apply(lambda x: len(x))
data = data[data['mash_len'] > 0]


### Identify and remove commonly used words in PRRs

In [ ]:
word_list = [y for x in list(data['mash']) for y in x]
counts = Counter(word_list)
Counter(word_list).most_common(50)

In [ ]:
Counter(word_list).most_common()[-50:-1]

In [191]:
common_list = ['report', 'request', 'record', 'city', 'please', 'copy', 'date', 'information', 'would',
              'include', 'document', 'provide']

In [193]:
# remove general words that are common to public record requests

for word in common_list:
    for i in range(len(data['mash'])):
        if word in data['mash'].iloc[i]:
            data['mash'].iloc[i].remove(word)

In [ ]:
# create column with the length of mash for each PRR

data['mash_len'] = data['mash'].apply(lambda x: len(x))

In [175]:
# remove entries of length 0

data = data[data['mash_len'] > 0]

In [176]:
data['mash_len'].describe()

count    73946.000000
mean        20.043451
std         36.064379
min          1.000000
25%          4.000000
50%         10.000000
75%         24.000000
max       2924.000000
Name: mash_len, dtype: float64

#### We can see a couple of examples of the cleaned mash and the original request:

In [177]:
data['mash'].iloc[7]

['build', 'permit', 'conversion', 'portion', 'garage', 'live', 'space']

In [180]:
data_raw["Summary"].iloc[7]

'Building permit from 2000 for the conversion of a portion of the garage to living space.'

In [194]:
data['mash'][86000]

['like',
 'build',
 'permit',
 'isnpection',
 'history',
 'certoificate',
 'occupancy',
 'muscovy',
 'rd',
 'specifically',
 'look',
 'apoproved',
 'final']

In [195]:
data_raw["Summary"].iloc[86000]

'I would like to request for building permit with isnpection history report or certoificate of occupancy for 1943 Muscovy Rd.\n\xa0\nI am specifically looking for apoproved final date information.'

In [201]:
data['report'] = data['mash'].apply(lambda x: "report" in x)

In [202]:
data_rep = data[data['report'] == True]

In [203]:
data_rep

,index,Summary,city,month_year,token,lemma,mash,mash_len,report
62,62,please send records the address greywalls dr ...,Arlington,2018-04,"[please, send, records, the, address, greywall...","[(please, VB), (send, NN), (records, NNS), (th...","[send, address, greywalls, dr, wa, family, squ...",24,True
90,90,incident report \ndate of incident \n\n any...,Arlington,2018-04,"[incident, report, date, of, incident, anyall,...","[(incident, JJ), (report, NN), (date, NN), (of...","[incident, incident, anyall, relate, andor, re...",33,True
93,93,i would like to request a copy of a police rep...,Arlington,2018-03,"[i, would, like, to, request, a, copy, of, a, ...","[(i, NN), (would, MD), (like, VB), (to, TO), (...","[like, police, file, accident, occur, january,...",31,True
102,102,my name is yazeed saeed g othman\ndriverís lic...,Arlington,2018-03,"[my, name, is, yazeed, saeed, g, othman, drive...","[(my, PRP$), (name, NN), (is, VBZ), (yazeed, V...","[name, yazeed, saeed, g, othman, driverís, lic...",42,True
141,141,all records files reports studies and correspo...,Arlington,2018-03,"[all, records, files, reports, studies, and, c...","[(all, DT), (records, NNS), (files, NNS), (rep...","[file, study, correspondence, relate, property...",98,True
164,164,police traffic collision report and any other ...,Arlington,2018-03,"[police, traffic, collision, report, and, any,...","[(police, NN), (traffic, NN), (collision, NN),...","[police, traffic, collision, police, report, r...",18,True
184,184,i am trying to obtain a copy of the police rep...,Arlington,2018-02,"[i, am, trying, to, obtain, a, copy, of, the, ...","[(i, NN), (am, VBP), (trying, VBG), (to, TO), ...","[try, obtain, police, duo, arrest, involve, re...",30,True
212,212,any records regarding death of sister danielle...,Arlington,2018-02,"[any, records, regarding, death, of, sister, d...","[(any, DT), (records, NNS), (regarding, VBG), ...","[regard, death, sister, danielle, gudde, witne...",12,True
213,213,all police reports charging documents and any ...,Arlington,2018-02,"[all, police, reports, charging, documents, an...","[(all, DT), (police, NN), (reports, NNS), (cha...","[police, charge, paperwork, report]",6,True
226,226,police report office erik moon responded to o...,Arlington,2018-02,"[police, report, office, erik, moon, responded...","[(police, NNS), (report, NN), (office, NN), (e...","[police, office, erik, moon, respond, zone, nd...",41,True


# LDA Analysis

In [196]:
# create dictionary and corpus
texts = list(data['mash'])
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [197]:
# 50 topics and 45 passes
lda_50_45_2 = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, 
                                         passes = 45, random_state=7)

In [188]:
# show topics for model
lda_50_45.show_topics(num_topics=50, formatted=False)

[(0,
  [('incident', 0.24412033),
   ('police', 0.09774184),
   ('involve', 0.08031427),
   ('occur', 0.05619594),
   ('july', 0.026982613),
   ('service', 0.025606353),
   ('mr', 0.025483835),
   ('person', 0.022150725),
   ('industrial', 0.019952968),
   ('avondale', 0.017577631)]),
 (1,
  [('list', 0.11351063),
   ('address', 0.089423396),
   ('business', 0.04984537),
   ('name', 0.046879493),
   ('january', 0.039695222),
   ('number', 0.033971623),
   ('email', 0.030741146),
   ('december', 0.029510023),
   ('month', 0.025103144),
   ('owner', 0.022423973)]),
 (2,
  [('request', 0.06177086),
   ('record', 0.059563518),
   ('public', 0.037712384),
   ('please', 0.022553125),
   ('act', 0.018288936),
   ('information', 0.015884787),
   ('pursuant', 0.014538696),
   ('california', 0.014509315),
   ('code', 0.013467154),
   ('provide', 0.012113321)]),
 (3,
  [('blvd', 0.09566692),
   ('nw', 0.08588162),
   ('washington', 0.06803973),
   ('point', 0.028145246),
   ('device', 0.017752174

In [198]:
# show topics for model
lda_50_45_2.show_topics(num_topics=50, formatted=False)

[(0,
  [('check', 0.10369195),
   ('see', 0.096796714),
   ('month', 0.03366025),
   ('account', 0.029787252),
   ('payee', 0.022412835),
   ('amount', 0.022256557),
   ('attachment', 0.022072963),
   ('escrow', 0.020911321),
   ('issue', 0.018710237),
   ('still', 0.018030368)]),
 (1,
  [('•', 0.12580512),
   ('east', 0.039862804),
   ('local', 0.038408782),
   ('emergency', 0.028766017),
   ('prevent', 0.024785917),
   ('hereby', 0.024181787),
   ('group', 0.021493912),
   ('impact', 0.019941604),
   ('nature', 0.01577148),
   ('replacement', 0.014921277)]),
 (2,
  [('fine', 0.04978936),
   ('february', 0.04836553),
   ('cam', 0.037812322),
   ('body', 0.036818836),
   ('injunction', 0.03467748),
   ('training', 0.024110764),
   ('stolen', 0.01653608),
   ('andrew', 0.015775861),
   ('lee', 0.015763437),
   ('harrison', 0.01561658)]),
 (3,
  [('lot', 0.105513096),
   ('collision', 0.08798288),
   ('blvd', 0.08653386),
   ('block', 0.072315335),
   ('south', 0.0612986),
   ('northeast